### classify stackoverflow posts using tf-idf features, using one-vs-all SVM

- note: use the same tokenizer rules as Keras

In [1]:
import csv
import os
import re
import sys
import numpy as np
import tensorflow as tf

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Input, Flatten, Activation, Lambda
from keras.layers import Conv1D, MaxPooling1D, Embedding, GlobalAvgPool1D
from keras.models import Model, Sequential

from sklearn import preprocessing, svm
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import cross_val_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.pipeline import Pipeline

# my stuff in the helpers/ directory
from helpers import files_helper, texts_helper, metrics_helper, tags_helper

Using TensorFlow backend.


In [2]:
SEED=np.random.randint(1,1000)
SEED

795

In [3]:
np.random.seed(seed=SEED)

In [4]:
texts, labels = files_helper.read_stackoverflow_sample_small()

In [5]:
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 1000
VALIDATION_SPLIT = 0.2
EMBEDDING_DIM=100
LABELS_MIN_DOC_COUNT = int(10)
BATCH_SIZE=32
NUM_EPOCHS=10

In [6]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS,
                     filters='\'!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
inverse_word_index = texts_helper.build_inverse_word_index(word_index)

In [7]:
[inverse_word_index[idx] for idx in sequences[0]]

['apply',
 'onclick',
 'event',
 'to',
 'an',
 'option',
 'i',
 'am',
 'using',
 'zend',
 'form',
 'to',
 'create',
 'a',
 'form',
 'i',
 'am',
 'also',
 'using',
 'mootools',
 'for',
 'javascript',
 'this',
 'gt',
 'radio',
 'alone',
 'array',
 'label',
 'gt',
 'are',
 'you',
 'going',
 'to',
 'be',
 'taking',
 'part',
 'with',
 'anyone',
 'else',
 'required',
 'gt',
 'true',
 'onclick',
 'gt',
 'gt',
 'array',
 'yes',
 'gt',
 'yes',
 'no',
 'gt',
 'no',
 'at',
 'the',
 'moment',
 'the',
 'onclick',
 'event',
 'works',
 'if',
 'any',
 'option',
 'is',
 'selected',
 'how',
 'do',
 'i',
 'get',
 'it',
 'to',
 'work',
 'for',
 'just',
 'yes',
 'being',
 'selected']

In [8]:
truncated_labels = tags_helper.truncate_labels(labels,LABELS_MIN_DOC_COUNT)

In [9]:
lb = preprocessing.MultiLabelBinarizer()
binary_labels = lb.fit_transform(truncated_labels)

In [10]:
# tag position => fraction of docs having that tag
tag_probabilities_index = tags_helper.get_probabilities_index(binary_labels)

In [11]:
tokenized_texts = []

for seq in sequences:
    truncated_seq = seq[:MAX_SEQUENCE_LENGTH]
    tokenized_txt = " ".join([inverse_word_index[idx] for idx in truncated_seq])
    tokenized_texts.append(tokenized_txt)

In [12]:
data = tokenized_texts
indices = np.arange(len(data))
np.random.shuffle(indices)

data = [data[i] for i in indices]
labels_1 = binary_labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * len(data))

X_train = data[:-num_validation_samples]
Y_train = labels_1[:-num_validation_samples]
X_val = data[-num_validation_samples:]
Y_val = labels_1[-num_validation_samples:]

len(X_train),len(X_val),Y_train.shape,Y_val.shape

(25600, 6400, (25600, 1209), (6400, 1209))

In [13]:
# bad order:

# clf = OneVsRestClassifier(Pipeline([
#     ('vect', CountVectorizer()),
#     ('tfidf', TfidfTransformer()),
#     ('clf', svm.LinearSVC()),
# ]), n_jobs=-1)


# good order (OVR just for the SVM, of course!)
clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', OneVsRestClassifier(svm.LinearSVC(),n_jobs=-1)),
])

parameters = [
    { 
          "clf__estimator__penalty": ["l2"],
          "clf__estimator__dual":[False,True],
          "clf__estimator__multi_class":["crammer_singer","ovr"],
          "clf__estimator__tol": [0.001],
          "clf__estimator__max_iter":[25,50,100],       
          "vect__max_features": [MAX_NB_WORDS]  
    },
    { 
          "clf__estimator__penalty": ["l1"],
          "clf__estimator__dual":[False],
          "clf__estimator__multi_class":["crammer_singer","ovr"],
          "clf__estimator__tol": [0.001],
          "clf__estimator__max_iter":[25,50,100],
          "vect__max_features": [MAX_NB_WORDS]  
    }    
]

In [14]:
best_score = float("-inf")

for g in ParameterGrid(parameters):
    clf.set_params(**g)
    clf.fit(X_train,Y_train)
    
    Y_pred_train = clf.predict(X_train)    
    Y_pred_val = clf.predict(X_val)
    
    train_score = f1_score(Y_train,Y_pred_train,average='micro')
    val_score = f1_score(Y_val,Y_pred_val,average='micro')
    
    current_score = val_score
    
    print("train micro-F1: {}".format(train_score))
    print("val micro-F1: {}".format(val_score))
    print("grid: {}".format(g))
    print("")
    
    if current_score > best_score:
        best_score = current_score
        best_grid = g

train micro-F1: 0.881497805514917
val micro-F1: 0.46662272855663306
grid: {'clf__estimator__dual': False, 'clf__estimator__tol': 0.001, 'vect__max_features': 20000, 'clf__estimator__multi_class': 'crammer_singer', 'clf__estimator__penalty': 'l2', 'clf__estimator__max_iter': 25}

train micro-F1: 0.9237511899843731
val micro-F1: 0.4399342804677684
grid: {'clf__estimator__dual': False, 'clf__estimator__tol': 0.001, 'vect__max_features': 20000, 'clf__estimator__multi_class': 'ovr', 'clf__estimator__penalty': 'l2', 'clf__estimator__max_iter': 25}

train micro-F1: 0.8814956109485537
val micro-F1: 0.46662272855663306
grid: {'clf__estimator__dual': False, 'clf__estimator__tol': 0.001, 'vect__max_features': 20000, 'clf__estimator__multi_class': 'crammer_singer', 'clf__estimator__penalty': 'l2', 'clf__estimator__max_iter': 50}

train micro-F1: 0.9237511899843731
val micro-F1: 0.4399342804677684
grid: {'clf__estimator__dual': False, 'clf__estimator__tol': 0.001, 'vect__max_features': 20000, 'clf_